In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table", "view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT *
FROM employee
WHERE title = "Sales Support Agent"


Done.


employee_id,last_name,first_name,title,reports_to,birthdate,hire_date,address,city,state,country,postal_code,phone,fax,email
3,Peacock,Jane,Sales Support Agent,2,1973-08-29 00:00:00,2017-04-01 00:00:00,1111 6 Ave SW,Calgary,AB,Canada,T2P 5M5,+1 (403) 262-3443,+1 (403) 262-6712,jane@chinookcorp.com
4,Park,Margaret,Sales Support Agent,2,1947-09-19 00:00:00,2017-05-03 00:00:00,683 10 Street SW,Calgary,AB,Canada,T2P 5G3,+1 (403) 263-4423,+1 (403) 263-4289,margaret@chinookcorp.com
5,Johnson,Steve,Sales Support Agent,2,1965-03-03 00:00:00,2017-10-17 00:00:00,7727B 41 Ave,Calgary,AB,Canada,T3B 1Y7,1 (780) 836-9987,1 (780) 836-9543,steve@chinookcorp.com


1. Write a query that returns each genre, with the number of tracks sold in the USA:
in absolute numbers
in percentages.
2. Write a paragraph that interprets the data and makes a recommendation for the three artists whose albums we should purchase for the store, based on sales of tracks from their genres.

In [13]:
%%sql
SELECT
    g.name,
    SUM(il.track_id) AS tracks_sold,
    p.percentage_sold
FROM genre AS g
INNER JOIN track AS t ON t.genre_id = g.genre_id
INNER JOIN invoice_line AS il ON il.track_id = t.track_id
INNER JOIN (
    SELECT 
        g.name,
        g.genre_id,
        il.quantity / il.* AS percentage_sold
    FROM genre AS g
    INNER JOIN track AS t ON t.genre_id = g.genre_id
    INNER JOIN invoice_line AS il ON il.track_id = t.track_id
    GROUP BY 1
    ) AS p ON p.genre_id = g.genre_id
GROUP BY 1
ORDER BY 3;

(sqlite3.OperationalError) near "*": syntax error
[SQL: SELECT
    g.name,
    SUM(il.track_id) AS tracks_sold,
    p.percentage_sold
FROM genre AS g
INNER JOIN track AS t ON t.genre_id = g.genre_id
INNER JOIN invoice_line AS il ON il.track_id = t.track_id
INNER JOIN (
    SELECT 
        g.name,
        g.genre_id,
        il.quantity / il.*  percentage_sold
    FROM genre AS g
    INNER JOIN track AS t ON t.genre_id = g.genre_id
    INNER JOIN invoice_line AS il ON il.track_id = t.track_id
    GROUP BY 1
    ) AS p ON p.genre_id = g.genre_id
GROUP BY 1
ORDER BY 3;]
(Background on this error at: http://sqlalche.me/e/13/e3q8)


# Comments


1. Write a query that finds the total dollar amount of sales assigned to each sales support agent within the company. Add any extra attributes for that employee that you find are relevant to the analysis.
2. Write a short statement describing your results, and providing a possible interpretation.

In [ ]:
%%sql



Your next task is to analyze the sales data for customers from each different country. You have been given guidance to use the country value from the customers table, and ignore the country from the billing address in the invoice table.

In particular, you have been directed to calculate data, for each country, on the:

total number of customers
total value of sales
average value of sales per customer
average order value
Because there are a number of countries with only one customer, you should group these customers as "Other" in your analysis. You can use the following 'trick' to force the ordering of "Other" to last in your analysis.

If there is a particular value that you would like to force to the top or bottom of results, you can put what would normally be your most outer query in a subquery with a case statement that adds a numeric column, and then in the outer query sort by that column. Here's an example - let's start by creating a view so we're working with a manageable number of rows:

In [14]:
%%sql
CREATE VIEW top_5_names AS
     SELECT
         first_name,
         count(customer_id) count
     FROM customer
     GROUP by 1
     ORDER by 2 DESC
     LIMIT 5;


SELECT * FROM top_5_names;

Done.
Done.


first_name,count
Frank,2
Mark,2
Aaron,1
Alexandre,1
Astrid,1


Next, inside a subquery, we'll select all values from our view and add a sorting column using a case statement, before sorting using that new column in the outer query.

In [15]:
%%sql
SELECT
    first_name,
    count
FROM
    (
    SELECT
        t5.*,
        CASE
            WHEN t5.first_name = "Mark" THEN 1
            ELSE 0
        END AS sort
    FROM top_5_names t5
   )
ORDER BY sort ASC

Done.


first_name,count
Frank,2
Aaron,1
Alexandre,1
Astrid,1
Mark,2


You should be able to adapt this technique into your query to force 'Other' to the bottom of your results. When working through this exercise, you will need multiple subqueries and joins. Imagine you work on a team of data analysts, and write your query so that it will be able to be easily read and understood by your colleagues.

1. Write a query that collates data on purchases from different countries.
Where a country has only one customer, collect them into an "Other" group.
The results should be sorted by the total sales from highest to lowest, with the "Other" group at the very bottom.
For each country, include:
total number of customers
total value of sales
average value of sales per customer
average order value

In [ ]:
%%sql


The Chinook store is setup in a way that allows customer to make purchases in one of the two ways:

purchase a whole album
purchase a collection of one or more individual tracks.
The store does not let customers purchase a whole album, and then add individual tracks to that same purchase (unless they do that by choosing each track manually). When customers purchase albums they are charged the same price as if they had purchased each of those tracks separately.

Management are currently considering changing their purchasing strategy to save money. The strategy they are considering is to purchase only the most popular tracks from each album from record companies, instead of purchasing every track from an album.

We have been asked to find out what percentage of purchases are individual tracks vs whole albums, so that management can use this data to understand the effect this decision might have on overall revenue.

It is very common when you are performing an analysis to have 'edge cases' which prevent you from getting a 100% accurate answer to your question. In this instance, we have two edge cases to consider:

Albums that have only one or two tracks are likely to be purchased by customers as part of a collection of individual tracks.
Customers may decide to manually select every track from an album, and then add a few individual tracks from other albums to their purchase.
In the first case, since our analysis is concerned with maximizing revenue we can safely ignore albums consisting of only a few tracks. The company has previously done analysis to confirm that the second case does not happen often, so we can ignore this case also.

In order to answer the question, we're going to have to identify whether each invoice has all the tracks from an album. We can do this by getting the list of tracks from an invoice and comparing it to the list of tracks from an album. We can find the album to compare the purchase to by looking up the album that one of the purchased tracks belongs to. It doesn't matter which track we pick, since if it's an album purchase, that album will be the same for all tracks.

Up until now, we've only ever compared two single values, using operators like = != and LIKE. To compare two tables of value, we can use the EXCEPT operator that we learned in the previous mission.

Let's say we had three tables in a database, as shown in the diagram below

We want to find a way to compare the letters columns from test_table_2, and test_table_3 to test_table_1 to see if they they are identical to test_table_1. Let's use EXCEPT with the two identical tables and see what we get with the first two tables:

Once we've made the comparison, we can wrap it in a CASE statement to add a column that tells us if that invoice was an album purchase or not.

You have everything you need to collate data on album vs single track purchases. This is easily the hardest query you have written so far, so take your time, and remember the query writing tips from the first screen!

1. Write a query that categorizes each invoice as either an album purchase or not, and calculates the following summary statistics:
Number of invoices
Percentage of invoices
2. Write one to two sentences explaining your findings, and making a prospective recommendation on whether the Chinook store should continue to buy full albums from record companies

In [ ]:
%%sql


In [ ]:
%%sql

In [ ]:
%%sql

In this guided project we put our SQL joining skills to the test, using subqueries, multiple joins, set operations, aggregate functions and more.

If you'd like more practice, you can look at the schema and come up with some more business questions, and then write queries to answer them. Here are a few to get you started:

Which artist is used in the most playlists?
How many tracks have been purchased vs not purchased?
Is the range of tracks in the store reflective of their sales popularity?
Do protected vs non-protected media types have an effect on popularity?
Curious to see what other students have done on this project? Head over to our Community to check them out. While you are there, please remember to show some love and give your own feedback!

And of course, we welcome you to share your own project and show off your hard work. Head over to our Community to share your finished Guided Project!

In [ ]:
%%sql

In [ ]:
%%sql

In [ ]:
%%sql